In [21]:

!curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip


SyntaxError: invalid syntax (<ipython-input-21-a2462e96cd8a>, line 1)

In [22]:
from google.colab import files
uploaded = files.upload()


In [23]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip') as zip_ref:
  zip_ref.extractall('data')

In [24]:
# import dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [25]:
print("The dimensions of movies dataframe are", movies_df.shape, '\nThe dimensions of ratings are:', ratings_df.shape)

The dimensions of movies dataframe are (9742, 3) 
The dimensions of ratings are: (100836, 4)


In [26]:
!pwd


/content


In [27]:
# find the head of the dataframes

print(movies_df.head(5))
print(ratings_df.head(5))


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [28]:
# Movie ID to movie name mappin
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())

print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full ratin matrix will have:", n_users*n_items, 'elements.')
print("..........")
print("Number of rating:", len(ratings_df))
print("Therefore:", len(ratings_df)/(n_users*n_items) * 100, '% of te matrix is filled.')
print("We ave an incredibly sparse matrix to work with")
print("And... as you can imagine, as the number users and items row, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work on a gobal scale... storing a full matrix will be a challenge")
print("one thing about matrix factorization is that we dont need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full ratin matrix will have: 5931640 elements.
..........
Number of rating: 100836
Therefore: 1.6999683055613624 % of te matrix is filled.
We ave an incredibly sparse matrix to work with
And... as you can imagine, as the number users and items row, the number of elements will increase by n*2
You are going to need a lot of memory to work on a gobal scale... storing a full matrix will be a challenge
one thing about matrix factorization is that we dont need all the data


In [30]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm.notebook import tqdm as tqdmnb

class MatrixFactorization(torch.nn.Module):
  def __init__(self, n_users, n_items, n_factors=20):
    super().__init__()
    # create user embeddings
    self.user_factors = torch.nn.Embedding(n_users, n_factors) # as look up table for input
    # create item embedding
    self.item_factors = torch.nn.Embedding(n_items, n_factors) # as look up table for input
    self.user_factors.weight.data.uniform_(0, 0.05)
    self.item_factors.weight.data.uniform_(0, 0.05)

  def forward(self, data):
    # matrix multiplication
    users, items = data[:,0], data[:,1]
    return (self.user_factors(users)*self.item_factors(items)).sum(1)

  def predict(self, user, item):
    return self.forward(user, item)


In [31]:
# creating a dataloader for pytorch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # elps transform data into machin learning readiness

# since data is already in mem we just roll with it
class Loader(Dataset):
  def __init__(self):
    self.ratings = ratings_df.copy()

    # extract all user Ids and movieids
    users = ratings_df.userId.unique()
    movies = ratings_df.movieId.unique()

    # Producing new continuous ids for movies and users


    # Unique values : index
    self.userid2idx = {o:i for i,o in enumerate(users)}
    self.movieid2idx = {o:1 for i,o in enumerate(movies)}

    # obtain continuous ID for user and movies
    self.idx2userid = {o:i for o,i in self.userid2idx.items()}
    self.idx2movieid = {o:i for i,o in self.movieid2idx.items()}

    # return te id from te indexed values as noted n lamda function below
    self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
    self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])

    self.x = self.ratings.drop(['rating', 'timestamp'], axis= 1).values
    self.y = self.ratings['rating'].values
    self.x = self.y = torch.tensor(self.x), torch.tensor(self.y) # transforms data into tensors for the torch models.

  def __getitem__(self, index):
    return (self.x[index], self.y[index])

  def __len__(self):
    return len(self.ratings)




In [32]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Runnin on GPU:", cuda)

model = MartixFactorization(n_users, n_items, n_factors= 8)
print(model)
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)

# GPU enable if you have one
if cuda:
  model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle= True)



Runnin on GPU: False
MartixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0195, 0.0310, 0.0080,  ..., 0.0282, 0.0287, 0.0138],
        [0.0348, 0.0046, 0.0014,  ..., 0.0468, 0.0136, 0.0152],
        [0.0017, 0.0405, 0.0325,  ..., 0.0330, 0.0144, 0.0059],
        ...,
        [0.0117, 0.0268, 0.0247,  ..., 0.0492, 0.0286, 0.0091],
        [0.0326, 0.0458, 0.0050,  ..., 0.0204, 0.0400, 0.0138],
        [0.0021, 0.0165, 0.0334,  ..., 0.0033, 0.0120, 0.0440]])
item_factors.weight tensor([[0.0255, 0.0186, 0.0034,  ..., 0.0117, 0.0397, 0.0450],
        [0.0113, 0.0317, 0.0257,  ..., 0.0440, 0.0326, 0.0179],
        [0.0319, 0.0086, 0.0075,  ..., 0.0385, 0.0499, 0.0419],
        ...,
        [0.0039, 0.0217, 0.0150,  ..., 0.0210, 0.0238, 0.0483],
        [0.0193, 0.0095, 0.0463,  ..., 0.0109, 0.0479, 0.0477],
        [0.0190, 0.0406, 0.0217,  ..., 0.0162, 0.0165, 0.0404]])


In [ ]:
for it in tqdmnb(range(num_epochs)):
  losses = []
  for x, y in train_loader:
    if cuda:
      x, y = x.cuda(), y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
  print("iter #()".format(it), "loss:", sum(losses) / len(losses))

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
C = 0
UW = 0
iw = 0
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)
    if C == 0:
      UW = param.data
      c +=1
    else:
      iw = param.data
      # print('param.data', param_data)

In [33]:
train_movie_embeddins = model.item_factors.weight.data.cpu().numpy()

In [34]:
len(train_movie_embeddins) # unique movie factor weights


9724

In [37]:
from sklearn.cluster import KMeans
# fit te clusters based on movie weights
kmeans = KMeans(n_clusters= 10, random_state= 0).fit(train_movie_embeddins)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [38]:
'''it can be seen here that the movies aare in the same cluster tend to have
similar genres, also note that the algorithm is unfarmiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #()".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId'] == movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #()


KeyError: 3